# Base Data Loader

## Introduction

Add a Markdown cell at the beginning of the notebook explaining the overall objective, including the use of LOAD CSV for importing data into Neo4j, and any prerequisites such as CSV file locations and database setup.

## Setup and Configuration

### local neo4j

Run the neo4j container with the data and import mount-points something like this

``` bash
docker run \
    --restart always \
    --publish=7474:7474 --publish=7687:7687 \
    --env NEO4J_AUTH=neo4j/password \
    --volume=/Volumes/DevStorage/Databases/neo4j/neo4jdata:/data \
    --volume=/Volumes/DevStorage/github/rlhatcher/ronaldhatcherblog/data/data_clean:/import \
    neo4j:5.16.0
```

The directory containing this notebook and the CSV data files should map to the import folder in the neo4j image.

### imports

In [1]:
from neo4j import GraphDatabase

### database connection

In [2]:
uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "password"))

## Queries

In [3]:
queries = {
    "clear_graph": "MATCH (n) DETACH DELETE n",
    "count_graph": "MATCH (n) RETURN COUNT(n)",
    "load_person": """
        CREATE (:Person {
            email: 'ronaldhatcher@mac.com',
            family_name: 'Hatcher',
            given_name: 'Ronald',
            id: 'kp_878e41a7d93846cca23114ad549e8126',
            picture: 'https://lh3.googleusercontent.com/a/ACg8ocKicc19o8E_h-mOGCLngSlTsFxJLcIGONz_ntc4RW5_3Q=s96-c'})
        """,
    "load_mfg": """
        LOAD CSV WITH HEADERS FROM $url AS line
        CREATE (:Manufacturer {name: line.name, id: line.id})
        """,
    "load_motor": """
        LOAD CSV WITH HEADERS FROM $url AS line
        MERGE (manufacturer:Manufacturer {id: line.mfgID})
        CREATE (motor:Motor {
          commonName: line.commonName,
          delays: line.delays,
          diameter: toFloat(line.diameter),
          infoUrl: line.infoUrl,
          totImpulseNs: toFloat(line.totImpulseNs),
          manufacturer: line.manufacturer,
          burnTimeS: toFloat(line.burnTimeS),
          propInfo: line.propInfo,
          length: toFloat(line.length),
          avgThrustN: toFloat(line.avgThrustN),
          dataFiles: line.dataFiles,
          impulseClass: line.impulseClass,
          sparky: line.sparky,
          caseInfo: line.caseInfo,
          propWeightG: toFloat(line.propWeightG),
          certOrg: line.certOrg,
          motorId: line.motorId,
          availability: line.availability,
          maxThrustN: toFloat(line.maxThrustN),
          totalWeightG: toFloat(line.totalWeightG),
          designation: line.designation,
          updatedOn: line.updatedOn,
          type: line.type
        })
        MERGE (manufacturer)-[:MAKES]->(motor)
        """,
    "load_kit": """
        LOAD CSV WITH HEADERS FROM $url AS row
        MERGE (manufacturer:Manufacturer {id: row.mfgID})
        CREATE (kit:Kit {
          id: row.UniqueID,
          name: row.Name,
          url: row.url,
          imageSrc: row['image_src'],
          recommendedEngines: row['Recommended Engines'],
          projectedMaxAltitude: row['Projected Max Altitude'],
          recoverySystem: row['Recovery System'],
          length: toFloat(row.Length),
          diameter:  toFloat(row.Diameter),
          estimatedWeight: toFloat(row['Estimated Weight']),
          estimatedAssemblyTime: row['Estimated Assembly Time'],
          finMaterials: row['Fin Materials'],
          decalType: row['Decal Type'],
          launchSystem: row['Launch System'],
          launchRodSize: row['Launch Rod Size'],
          instructions: row.instructions,
          ageRecommendation: row['Age Recommendation'],
          height: toFloat(row.Height),
          weight: toFloat(row.Weight),
          motorMount: row['Motor Mount'],
          parachuteSize: row['Parachute Size'],
          shockCordType: row['Shock Cord Type'],
          shockCordMount: row['Shock Cord Mount'],
          finThickness: toFloat(row['Fin Thickness']),
          ringThickness: toFloat(row['Ring Thickness']),
          instructionsUrl: row.Instructions,
          price: toFloat(row.Price),
          currency: row.Currency,
          sku: row.SKU,
          stockStatus: row['Stock Status'],
          description: row.Description,
          links: row.Links,
          parachute: row.Parachute,
          finArray: row['Fin Array'],
          complexity: NULL
        })
        MERGE (manufacturer)-[:MAKES]->(kit);
        """,
    "load_rocket": """
        LOAD CSV WITH HEADERS FROM $url AS line
        MERGE (manufacturer:Manufacturer {id: line.mfgID})
        CREATE (rocket:Rocket {
          id: line.id,
          name: line.name,
          descripton: line.description,
          image: line.image
        })
        MERGE (manufacturer)-[:MAKES]->(rocket);
        """,
    "load_model": """
        LOAD CSV WITH HEADERS FROM $url AS line
        CREATE (:Model:Rocket {name: line.name, id: line.id, image: line.image})
        """,
    "load_design": """
        LOAD CSV WITH HEADERS FROM $url AS line
        CREATE (:Design 
          { id: line.id, 
            rocketId: line.rocketId, 
            name: line.name, 
            filename: line.filename, 
            stages: toInteger(line.stages),
            massEmpty: toFloat(line.massEmpty),
            stabilityCal: toFloat(line.stabilityCal),
            stabilityPct: toFloat(line.stabilityPct),
            cg: toFloat(line.cg),
            cp: toFloat(line.cp),
            length: toFloat(line.length),
            maxDiameter: toFloat(line.maxDiameter)
          })
        """,
    "vorb_lone": """
        MATCH (r:Rocket {id: 'a44238b2-9ed1-4500-9112-08d79009ab49'}), (m:Model {id: 'd3cd5b35-924f-405f-bbd7-9c435a018453'})
        MERGE (m)-[:BASED_ON]->(r)
        """,
    "hughes_aim54": """
        MATCH (r:Rocket {id: 'fd2a6b9a-a9b5-4efe-95d2-c028171b8767'}), (m:Model {id: 'b5294a38-eaf5-4b7d-a6ae-de4468d9c9e0'})
        MERGE (m)-[:BASED_ON]->(r)
        """,
    "aim54_kit": """
        MATCH (k:Kit {id: '9e295916-5309-4471-847d-4d42c814a806'}), (m:Model {id: 'b5294a38-eaf5-4b7d-a6ae-de4468d9c9e0'})
        MERGE (m)-[:BASED_ON]->(k)
        """,
    "person_models": """
        MATCH (p:Person {id: "kp_878e41a7d93846cca23114ad549e8126"}), (m:Model)
        MERGE (p)-[:OWNS]->(m)
        """
}

## Query Execution

In [4]:
def execute_query(driver, query_key, parameters=None):
    with driver.session() as session:
        result = session.execute_write(lambda tx: tx.run(queries[query_key], parameters))
        return result

## Data Loading

In [5]:
execute_query(driver, "clear_graph")

# Nodes
execute_query(driver, "load_mfg", {"url": "file:///manufacturers.csv"})
execute_query(driver, "load_motor", {"url": "file:///motors.csv"})
execute_query(driver, "load_kit", {"url": "file:///kits.csv"})
execute_query(driver, "load_rocket", {"url": "file:///rockets.csv"})
execute_query(driver, "load_person")
execute_query(driver, "load_model", {"url": "file:///models.csv"})

# Hacks
execute_query(driver, "vorb_lone")
execute_query(driver, "hughes_aim54")
execute_query(driver, "aim54_kit")
execute_query(driver, "person_models")

## Validation

After loading the data, include steps to validate the import process (e.g., count nodes and relationships).

```python
# Example validation query
result = execute_query(driver, "count_people")  # Assume you have defined this query
for record in result:
    print("Number of people:", record["count"])
```

Add a concluding Markdown cell summarizing the work done and any observations or next steps.

## Cleanup

In [6]:
driver.close()